# Funciones y decoradores

Hemos visto que en Python todo es un objeto, con lo cual, incluso las funciones, son objetos. Como tales tienen métodos y atributos:

In [ ]:
lio_messi = "Lio Messi"
print(type(lio_messi))

La variable `lio_messi` es un string, y como tal, pertenece a la clase `str`, que tiene sus propios atributos y métodos:

Veamos qué pasa con las funciones:

In [ ]:
def saluda_a(alguien):
    saludo = f"Hola {alguien}!"
    return saludo

In [ ]:
print(saluda_a(lio_messi))

In [ ]:
print(type(saluda_a))

> Un atributo interesante de las funciones es `__name__` por razones que veremos en breve:

In [ ]:
print(saluda_a.__name__)

Es decir, `__name__` es el nombre de la función, que está guardado dentro del objeto que representa dicha función.
> La capacidad del lenguaje de responderse preguntas sobre las propias entidades que componen el lenguaje se llama _introspección_. 

En la clase anterior vimos dos características importantes de las funciones en Python. La primera de ellas es que las funciones pueden retornar (esto es, crear) otras funciones:

In [ ]:
def genera_recta(a,b):
    "Genera la función recta y = a x + b"
    def recta(x):
        "Evalúa la función recta en x"
        y = a * x + b
        return y
    return recta

In [ ]:
f = genera_recta(2,3)       # f(x) = 2 * x + 3
x = 2
print(f"f({x}) = {f(x)}")   # f(2) = 2 * 2 + 3 
x = 0
print(f"f({x}) = {f(x)}")   # f(0) = 2 * 0 + 3 

In [ ]:
print(type(f))

La segunda de ellas es que es posible pasar como argumento una función a otra:

In [ ]:
g = genera_recta(1,-1) # g(x) = x - 1
x = 3
y = f(g(x))
print(f"y = {y}") 

In [ ]:
print(type(g))

## Funciones que aceptan y devuelven funciones (Decoradores)


Vamos a trabajar ahora con los decoradores. Los decoradores no son otra cosa que funciones, pero que, por sus características, adquieren ese nombre y una forma particular de llamarlos que reduce convenientemente la sintaxis al programar. Empecemos por definir una función que devuelve otra función, como vimos arriba, de la siguiente forma:

In [ ]:
def mi_decorador(func):
    def wrapper():
        print(f"Por llamar a la función {func.__name__}")
        func()
        print(f"Listo, ya llamé a la función {func.__name__}")
    return wrapper


Definamos ahora un saludo genérico:

In [ ]:
def saluda():
    print("Holaa!!")

In [ ]:
saluda()

Nada nuevo hasta ahora, pero empecemos a combinar las funciones:

In [ ]:
saluda_w = mi_decorador(saluda)

In [ ]:
saluda_w()

In [ ]:
print(type(saluda_w))

Tenemos ahora una función `saluda` y su versión _decorada_ `saluda_w`, que simplemente llama a la función `saluda`, pero además imprime mensajes antes y después del llamado a la función. Esto es algo que uno va a querer hacer, por ejemplo para calcular el tiempo de ejecución de una función, o para imprimir mensajes de registro (_logging_) o debug, u otras tantas cosas más. Por eso Python introduce una notación especial para este tipo de funciones `mi_decorador`:

In [ ]:
saluda = mi_decorador(saluda)

In [ ]:
@mi_decorador
def saluda_en_ingles():
    print("Hello!!")

> Notar que el decorador siempre empieza con el símbolo `@` y se encuentra en la línea inmediatamente anterior a la definición de la función.

In [ ]:
saluda_en_ingles()

Qué pasa si queremos aplicar el decorador a una función que recibe argumentos como `saluda_a`?

In [ ]:
@mi_decorador
def saluda_a(alguien):
    print(f"Hola {alguien}!")

In [ ]:
saluda_a("Lio Messi")

Notemos que como está definido el decorador, recibe una función sin argumentos:
```Python
def mi_decorador(func):
    def wrapper():
        print(f"Por llamar a la función {func.__name__}")
        func()
        print(f"Listo, ya llamé a la función {func.__name__}")
    return wrapper
```


En este último caso, al aplicar `@mi_decorador` a `saluda_a(alguien)`, estamos pasando a la función `mi_decorador` una función `func` que dentro de `mi_decorador` se llama como `func()`, es decir, no tiene argumentos. Para resolver este problema, tenemos que indicar explícitamente que la función que vamos a llamar dentro del decorador puede tener argumentos: 

In [ ]:
def mi_nuevo_decorador(func):
    def wrapper(*args, **kwargs):
        print(f"Por llamar a la función {func.__name__}")
        func(*args, **kwargs)
        print(f"Listo, ya llamé a la función {func.__name__}")
    return wrapper

Hasta ahora la función `func` que envuelve el decorador no devuelve ningún valor, sólo imprime un mensaje en pantalla. Cómo hacemos para usar un decorador con una función que devuelve un valor?

In [ ]:
def proto_debug_decorator(func):
    def wrapper(*args, **kwargs):
        print(f"Por llamar a la función {func.__name__}")
        resultado = func(*args, **kwargs)
        print(f"Listo, ya llamé a la función {func.__name__}")
        return resultado
    return wrapper

In [ ]:
@proto_debug_decorator
def mi_calculo_complicado(x,y,z=0):
    return x**2 + y**2 + z**2
    

In [ ]:
v = mi_calculo_complicado(1,2,3)
print(v)

## Decoradores, un ejemplo más útil

Recordemos que al llamar una función, `*args` representa a la tupla de argumentos mientras que `**kwargs` es el diccionario de argumentos opcionales. Escribamos un par de funciones útiles para transformar estos tipos en string, de modo que se puedan imprimir, por ejemplo:

In [ ]:
def args_as_str(*args, **kwargs):
    args_str = ", ".join([str(a) for a in args])
    kwargs_str = ", ".join([f"{k}={v}" for k,v in kwargs.items()])
    return f"{args_str}, {kwargs_str}"

In [ ]:
args_as_str(1,3,hola="Hello", a  =  5 )

In [ ]:
def debug_me(func):
    def wrapper(*args, **kwargs):
        print(f"{func.__name__} ({args_as_str(*args, **kwargs)})")
        resultado = func(*args, **kwargs)
        print(f"Listo, ya llamé a la función {func.__name__}")
        return resultado
    return wrapper

In [ ]:
@debug_me
def mi_calculo_recontracomplicado(x,y,z=0):
    return x**2 + y**2 + z**2

In [ ]:
v =  mi_calculo_recontracomplicado(1,2,z=3)

In [ ]:
print(v)

----

## Ejercicios 05 (c)

6. El módulo time calcula el tiempo en segundos desde el comienzo de la era de la computación (?), que para los fines prácticos, da inicio el 1 de enero de 1970 ;-D. Veamos unos ejemplos de su uso:

In [ ]:
import time 

ahora = time.time()
print (ahora)
# duerme 5 segundos
time.sleep(5) # zzzz.....

ahora = time.time()
print (ahora)

Utilizando las funciones anteriores, escriba el decorador `@time_me` que calcula e imprime el tiempo que tarda en 
ejecutarse una función. **No empiece desde cero!!** Use como plantilla para empezar el decorador `@debug_me` y modifíquelo adecuadamente.   

In [ ]:
# descomente el decorador una vez que lo tenga programado
# @time_me
def mi_calculo_recontralargo(n):
    l= [x for x in range(n)]
    return sum(l)

In [ ]:
mi_calculo_recontralargo(20000000)


----
